In [ ]:
#| label: libraries
#| include: false

import pandas as pd
import numpy as np
import plotly.express as px

## Elevator pitch

_We can predict if a house sold in Denver, CO in 2013 was built before or after 1980 with a machine learning model. We can measure the accuracy score of the model to see it's effectiveness at predicting this._


In [ ]:
#| label: project-data

url = "https://raw.githubusercontent.com/byuidatascience/data4dwellings/master/data-raw/dwellings_ml/dwellings_ml.csv"

dwellings_ml = pd.read_csv(url)

# dwellings_ml.head()

## QUESTION|TASK 1

__Create 2-3 charts that evaluate potential relationships between the home variables and before1980. Explain what you learn from the charts that could help a machine learning algorithm.__

_I created 3 charts comparing the relationship between `Year Built` and `Sell Price`, `Number of Bathrooms`, and `Number of Bedrooms`. These comparisons help identify data trends useful for training the model._


In [ ]:
#| label: Q1

# relationship between 'yrbuilt' (year built) and 'sprice' (sell price)
fig1 = px.scatter(
  dwellings_ml, 
  x='yrbuilt', 
  y='sprice', 
  color='sprice', 
  title='Sell Price and Year Built', 
  labels={
    'yrbuilt': 'Year Built', 
    'sprice': 'Sell Price'
    }
  )


# relationship between 'yrbuilt' and 'numbaths' (number of bathrooms)
baths_count = dwellings_ml.groupby(['yrbuilt', 'numbaths']).size().reset_index(name='count')

fig2 = px.bar(
  baths_count, 
  x='yrbuilt', 
  y='count', 
  color='numbaths', 
  title='Number of Bathrooms and Year Built',
  labels={
    'yrbuilt': 'Year Built', 
    'count': 'Number of Houses', 
    'numbaths': 'Number of Bathrooms'
    }
  )


# relationship between 'yrbuilt' and the 'numbdrm' (number of bedrooms)
bedrooms_count = dwellings_ml.groupby(['yrbuilt', 'numbdrm']).size().reset_index(name='count')

fig3 = px.bar(
  bedrooms_count, 
  x='yrbuilt', 
  y='count', 
  color='numbdrm', 
  title='Number of Bedrooms and Year Built',
  labels={
    'yrbuilt': 'Year Built', 
    'count': 'Number of Houses', 
    'numbdrm': 'Number of Bedrooms'
    }
  )


fig1.show()
fig2.show()
fig3.show()

## QUESTION|TASK 2

__Build a classification model labeling houses as being built “before 1980” or “during or after 1980”. Your goal is to reach or exceed 90% accuracy. Explain your final model choice (algorithm, tuning parameters, etc) and describe what other models you tried.__

_I tested a Decision Tree and Random Forest Classifier. By dropping irrelevant columns (`parcel`, `yrbuilt`) and optimizing the dataset, the Random Forest model seemd to perform better._


In [ ]:
#| label: Q2

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# columns to drop to prepare training and test data
features_to_drop = ['parcel', 'abstrprd', 'before1980', 'yrbuilt']
X = dwellings_ml.drop(columns=features_to_drop)
y = dwellings_ml['before1980']

# splitting dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# initialize and train the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train)

# predict on test set
y_pred = dt_classifier.predict(X_test)

# W10 P4 Quiz Question #1
# average_first_10_test_y = y_test.iloc[:10].mean()
# print(f"{average_first_10_test_y}\n")

# W10 P4 Quiz Question #2
# average_first_10_training_X_sprice = X_train.iloc[:10]['sprice'].mean()
# print(f" Average of the first 10 values in training X values for the selling price (sprice): {average_first_10_training_X_sprice}\n")

# calculate accuracy as percentage
accuracy = accuracy_score(y_test, y_pred)
print(f"DTC Model Accuracy: {accuracy * 100:.2f}%")

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# initialize Random Forest Classifier model
rf_classifier = RandomForestClassifier(random_state=42)

# train model on training data
rf_classifier.fit(X_train, y_train)

# predict on testing data
y_pred_rf = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_rf)
print(f"RFC Model Accuracy: {accuracy * 100:.2f}%")

## QUESTION|TASK 3

__Justify your classification model by discussing the most important features selected by your model. This discussion should include a feature importance chart and a description of the features.__

_The Random Forest model identified key predictors like location (`are`), architectural style (e.g., `1-story`), and the number of bathrooms. These features were most impactful, achieving **92%** accuracy._


In [ ]:
#| label: Q3

# extract feature importances from model
feature_importances = rf_classifier.feature_importances_

# create dataframe for visualization
features_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# sort dataframe by importance
features_df = features_df.sort_values(by='Importance', ascending=False)

# visualizing most important features
fig = px.bar(
  features_df.head(), 
  x='Importance', 
  y='Feature', 
  orientation='h',
  title='5 Most Important Features in Predicting Year Built',
  labels={
    'Feature': 'Feature', 
    'Importance': 'Importance Score'
    }
  )

fig.show()

## QUESTION|TASK 4

__Describe the quality of your classification model using 2-3 different evaluation metrics. You also need to explain how to interpret each of the evaluation metrics you use.__

_I used Precision, Recall, and AUROC scores to evaluate the model. Precision measures the accuracy of predictions, Recall identifies the model's sensitivity to 'before 1980' houses, and AUROC indicates overall classification performance._


In [ ]:
#| label: Q4

from sklearn.metrics import precision_score, recall_score, roc_auc_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auroc = roc_auc_score(y_test, y_pred)

print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"AUROC: {auroc * 100:.2f}%")